# Using OpenAI Chat Completions

This notebook covers how to use the Chat Completions API and other features such as creating prompts and function calling.


## Instantiating the OpenAI Client

The OpenAI client object is used to get responses from the API. This will automatically read your API and org key from your environment variables.

You can optionally pass in your API key and org key as arguments: `api_key` and `organization`.


In [1]:
from not_again_ai.llm.chat_completion.providers.openai_api import openai_client

client = openai_client()

## Basic Chat Completion

The `chat_completion` function is an easy way to get responses from OpenAI models.
It requires the prompt to the model to be formatted in the chat completion format,
see the [API reference](https://platform.openai.com/docs/api-reference/chat/create) for more details.


In [2]:
from not_again_ai.llm.chat_completion import chat_completion
from not_again_ai.llm.chat_completion.types import ChatCompletionRequest, SystemMessage, UserMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="Hello!"),
]
request = ChatCompletionRequest(
    messages=messages,
    model="gpt-4o-mini-2024-07-18",
    max_completion_tokens=100,
)
response = chat_completion(request, "openai", client)

response.choices[0].message.content

'Hello! How can I assist you today?'

## Creating Prompts

Injecting variables into prompts is a common task and we provide the `chat_prompt` which uses [Liquid templating](https://jg-rp.github.io/liquid/).

In the `messages` argument, the "content" field can be a [Python Liquid](https://jg-rp.github.io/liquid/introduction/getting-started) template string to allow for more dynamic prompts which not only supports variable injection, but also conditional logic, loops, and comments.


In [3]:
from not_again_ai.llm.prompting.compile_prompt import compile_messages

place_extraction_prompt = [
    SystemMessage(
        content="""- You are a helpful assistant trying to extract places that occur in a given text.
- You must identify all the places in the text and return them in a list like this: ["place1", "place2", "place3"]."""
    ),
    UserMessage(
        content="""Here is the text I want you to extract places from:
{%- # The user's input text goes below %}
{{text}}""",
    ),
]

variables = {
    "text": "I went to Paris and Berlin.",
}

messages = compile_messages(messages=place_extraction_prompt, variables=variables)
messages

[SystemMessage(content='- You are a helpful assistant trying to extract places that occur in a given text.\n- You must identify all the places in the text and return them in a list like this: ["place1", "place2", "place3"].', role=<Role.SYSTEM: 'system'>, name=None),
 UserMessage(content='Here is the text I want you to extract places from:\nI went to Paris and Berlin.', role=<Role.USER: 'user'>, name=None)]

## Token Management

While the OpenAI chat completion will return the tokens used, the `num_tokens_from_messages` helper can be used to compute the number of tokens used in a list of messages before calling the API.

We explicitly require a tokenizer since loading it has some overhead, so we want to avoid doing so many times for certain use cases.

NOTE: This function not support counting tokens used by function calling.


In [4]:
from not_again_ai.llm.prompting.providers.openai_tiktoken import TokenizerOpenAI

tokenizer = TokenizerOpenAI(model="gpt-4o-mini-2024-07-18")
num_tokens = tokenizer.num_tokens_in_messages(messages=messages)
print(num_tokens)

78


## Chat Completion with Function Calling and other Parameters

The `chat_completion` function can also be used to call functions in the prompt and a myriad of other commonly used parameters like temperature, max_tokens, and logprobs. See the docstring for more details.

See the [gpt-4-v.ipynb](gpt-4-v.ipynb) for full details on how to use the vision features of `chat_completion` and `chat_prompt`.


In [5]:
# Define a tool to get the current weather
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        },
    },
]
# Ask the model to call the function
messages = [
    UserMessage(
        content="What's the current weather like in {{ city_state }} today? Call the get_current_weather function.",
    )
]

messages = compile_messages(messages=messages, variables={"city_state": "Boston, MA"})

client = openai_client()

request = ChatCompletionRequest(
    messages=messages,
    model="gpt-4o-mini-2024-07-18",
    client=client,
    tools=tools,
    tool_choice="required",  # Force the model to use the tool
    max_completion_tokens=300,
    temperature=0,
    log_probs=True,
    top_log_probs=2,  # returns the log probabilities of the top 2 tokens
    seed=42,  # Set the seed for reproducibility. The API will also return a `system_fingerprint` field to monitor changes in the backend.
    n=2,  # Generate 2 completions at once
)
response = chat_completion(request, "openai", client)
response

ChatCompletionResponse(choices=[ChatCompletionChoice(message=AssistantMessage(content='', role=<Role.ASSISTANT: 'assistant'>, name=None, refusal=None, tool_calls=[ToolCall(id='call_dwyBECUXUbPyuJH6oGxE3DFz', function=Function(name='get_current_weather', arguments={'location': 'Boston, MA', 'format': 'fahrenheit'}), type='function')]), finish_reason='tool_calls', json_message=None, logprobs=None, extras={}), ChatCompletionChoice(message=AssistantMessage(content='', role=<Role.ASSISTANT: 'assistant'>, name=None, refusal=None, tool_calls=[ToolCall(id='call_yB03eV0flHXXKI6STtHMvPpm', function=Function(name='get_current_weather', arguments={'location': 'Boston, MA', 'format': 'fahrenheit'}), type='function')]), finish_reason='tool_calls', json_message=None, logprobs=None, extras={})], errors='', completion_tokens=46, prompt_tokens=99, completion_detailed_tokens=None, prompt_detailed_tokens=None, response_duration=0.8277, system_fingerprint='fp_e4fa3702df', extras={'prompt_filter_results': N